In [1]:
import numpy as np
from oe_acute import MNE
from oe_acute import trial_utils as tu
import os
import pickle
import glob

In [2]:
import numpy as np
from scipy.io.wavfile import read
from scipy import signal
from scipy.optimize import minimize
import glob
import os
import pickle
from oe_acute import trial_utils as tu
import tqdm
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import networkx as nx

In [3]:
exp_path = '/mnt/sphere/share/NCM_acute/experiments/B1240/block3/'
sort_path =  '/mnt/sphere/share/NCM_acute/sorting/B1240/block3/'
stim_path = '/mnt/sphere/share/NCM_acute/MET_prediction_expt/'

ratings = [1,2,3]

ratings_str = ''.join([str(x) for x in ratings])

In [4]:
nperseg=128
noverlap=64
nfft=128
nsegbins=64
bp_low=300
bp_high=12000
bp_order=5
preemph_b=0.97
ref_level_dB=20
min_level_dB=-30

spec_all = []
stereo_all = []
stim_files = glob.glob(os.path.join(stim_path, '*.wav.sine'))

In [5]:
n_rows = 0
n_cols = 2
nsegbins = 64

In [6]:
stim_spec1, stim_resp1 = MNE.prepare_stim_resp_dset(exp_path, sort_path, stim_path, stim_preprocess_func=MNE.preprocess_wav_AVGN, ratings=ratings, avg_nrows=n_rows, avg_ncols=n_cols)

3_E6211_s_13@1___E6211_s_13@1


KeyboardInterrupt: 

In [6]:
#3_E6211_s_13@1___E6211_s_13@1
spec_f = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/stim_spec.pkl'
with open(spec_f, 'rb') as f:
    stim_spec = pickle.load(f)

In [7]:
import librosa
dur = librosa.get_duration(filename='/mnt/sphere/share/NCM_acute/MET_prediction_expt/3_E6211_s_13@1___E6211_s_13@1.wav')
#3_E6211_s_13@1___E6211_s_13@1.wav

In [8]:
dur

3.9012244897959185

In [11]:
np.shape(stim_spec[0])

(64, 365)

In [13]:
spec_f = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/stim_spec.pkl'
with open(spec_f, 'wb') as f:
    pickle.dump(stim_spec, f)

resp_f = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/stim_resp.pkl'
with open(resp_f, 'wb') as f:
    pickle.dump(stim_resp, f)

### Load stim_spec, stim_resp

In [6]:
spec_f = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/stim_spec.pkl'
with open(spec_f, 'rb') as f:
    stim_spec = pickle.load(f)

resp_f = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/stim_resp.pkl'
with open(resp_f, 'rb') as f:
    stim_resp = pickle.load(f)

In [7]:
type(stim_spec), type(stim_resp)

(list, list)

In [17]:
#dset_segs = MNE.prepare_AE_dset(stim_spec, stim_resp, nsegbins=nsegbins, thresh=0.01)

Final shape: (60430, 278, 64)


In [9]:
dsegs, spec1 = MNE.prepare_temporal_dset(stim_spec, stim_resp, nsegbins=nsegbins, thresh=0.01)

Final shape: (60429, 278, 64)
Next shape: (60429, 278)


### Reduce to 16 bins

In [10]:
np.shape(stim_spec[0]), np.shape(stim_resp[0])

((64, 365), (214, 365))

In [16]:
stim_spec_16 = []
for stim in stim_spec:
    stim_16 = MNE.kozlov_averaging(stim, n_rows=n_cols, n_cols=n_rows)
    stim_spec_16.append(stim_16)

In [18]:
nsegbins1 = 16

In [19]:
dsegs_16, spec1_16 = MNE.prepare_temporal_dset(stim_spec_16, stim_resp, nsegbins=nsegbins1, thresh=0.01)

Final shape: (54283, 230, 16)
Next shape: (54283, 230)


In [20]:
spec_next_name = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/next_list.pkl'
with open(spec_next_name, 'wb') as f:
    pickle.dump(spec1, f)

spec_segs_name = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/segs_list.pkl'
with open(spec_segs_name, 'wb') as f:
    pickle.dump(dsegs, f)

OverflowError: cannot serialize a bytes object larger than 4 GiB

In [21]:
spec_next_name = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/next_list_16.pkl'
with open(spec_next_name, 'wb') as f:
    pickle.dump(spec1_16, f)

spec_segs_name = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/segs_list_16.pkl'
with open(spec_segs_name, 'wb') as f:
    pickle.dump(dsegs_16, f)